<img align="right" src="tf-small.png"/>

# tfQuery

Do we need a query language in TF, like MQL?

Yes, it is convenient to have a more declarative way of getting a set of interesting nodes to work with.
But should it be MQL?

Experience shows that MQL may give you a very good first try, 
until you realize that you may not have queried for all cases.
You forgot to query for some elements in a different order.
You have not reckoned with gaps.
And the query does not give you interesting things from the context with the results.
Also, MQL does not work nicely with object types that are scattered through other object types, such as the
[*lexeme*](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/otype.html)
type.

Look in SHEBANQ for examples how unwieldy MQL queries may become.

Here is a good example:

[Dirk Roorda: Yesh](https://shebanq.ancient-data.org/hebrew/query?version=4b&id=556)

```
select all objects where
[book [chapter [verse
[clause
    [clause_atom
        [phrase
            [phrase_atom
                [word focus lex="JC/" OR lex=">JN/"]
            ]
        ]
    ]
]
]]]
```

Well, this is not too complicated, but the query misses results.
See [here](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/0_mql.html)
to see what would be needed to make it right.

Also, I wonder: do we want a new language?
Suppose we make a TFQL, then we need a parser for it,
we need to define a syntax, we need to refine the syntax, update the parser, etc.
It will become a cumbersome straight-jacket.

In our case, we do not have the requirement that non-coders should be able to use TFQL in a stand-alone manner.

On the contrary, TFQL should live in a programming environment, and we can take advantage of that.

Here are initial thought for **tfQuery**, a query *mechanism* inside TF, not a *language*.

* tfQuery defines queries as data structures in Python, more precisely: as a graph
* it does not matter how you build up a query, tfQuery processses the value of a datastructure
  that you pass to it. The surface syntax will not be seen by tfQuery
* a query is a graph representation where the nodes are things like
  
  `('phrase', dict(det='und'))`
  
  or
  
  `('word', dict(sp='verb', gn='f', ps='3f'))`

* the edges specify relations between the nodes, like: *is contained in*, *follows*,
  *precedes*
  
In MQL you also specify a graph, by means of a template, but this template forces you to *overspecify*: the template often implies more constraints then you really want.

So how do we specify edges? As constraints.

Let us formulate a query for

* clauses that are object clauses
* containing two phrases (both undetermined)
* one of which contains a verb in the third person feminine
* and the other phrase contains a feminine, plural noun

In MQL

```
[clause rela='Objc'
    [phrase det='und'
        [word sp='verb' AND gn='f' AND ps='p3']
    ]
    [phrase det='und'
        [word sp='subs' AND gn='f' AND nu='pl']
    ]
]
```

Here is how we are going to do it,
and note that we are going to write executable code!

In [1]:
c = ('clause', dict(rela='Objc'))
p1 = ('phrase', dict(det='und'))
p2 = ('phrase', dict(det='und'))
w1 = ('word', dict(sp='verb', gn='f', ps='p3'))
w2 = ('word', dict(sp='subs', gn='f', nu='pl'))

In [2]:
nodes = [c, p1, p2, w1, w2]
edges = [
    (c, [p1,p2]),
    (p1, [w1]),
    (p2, [w2]),
    (p1, p2),
]

query = (nodes, edges)

An edge of like `(x, [y,z])` means that `y` and `z` are embedded in `x`, but does not mean
that `y` comes before `z`.

An edge like `(x, y)` means that `x` comes before `y`.

## Increased flexibility

Note that it is very easy to remove the `(p1, p2)` condition, which states that the first
phrase comes before the second one.

If we wanted to do that in MQL, the query would become:

```
[clause rela='Objc'
    [phrase det='und'
        [word sp='verb' AND gn='f' AND ps='p3']
    ]
    [phrase det='und'
        [word sp='subs' AND gn='f' AND nu='pl']
    ]
    OR
    [phrase det='und'
        [word sp='subs' AND gn='f' AND nu='pl']
    ]
    [phrase det='und'
        [word sp='verb' AND gn='f' AND ps='p3']
    ]
]
```

This goes quickly out of hand, see e.g.
[Dirk Roorda: Object clauses of verbless mothers](https://shebanq.ancient-data.org/hebrew/query?id=984) and accompanying
[notebook](https://shebanq.ancient-data.org/shebanq/static/docs/tools/shebanq/VerblessMothers.html)

## Query results

What should we return as query results?
Do we want every instantiation of the nodes that satisfy the criteria?

That can become overwhelming. 
If for example you search for a word in a book, an other word in the same book, and a third word in the same book without further constraints, then for a book with 10,000 words you'll get 10,000 * 10,000 * 10,000 results or 1 Tera results, which is, even for a computer, a bit much.

This is why Ulrik invented the sheaf.

Our way of solving this problem could be like this:

* we return a collection of node lists: for each node in the query we return the 
  corresponding node list;
* these lists consist of TF nodes which are guaranteed to occur in at least one
  instantatiation of the whole graph;
* we return nothing else.

It is up to the user to pick one of these nodesets and to further process them.
If he needs needs context around the result nodes, he can easily draw the info from there.

It is even possible to generate the code to get full results from the original query graph.

So the real result is two things:

* a collection of node lists
* a function to look up other result nodes in the context of a given result node.

## Implementation

How could we implement this search efficiently?

First idea:

* build for each query graph node
  (which corresponds to a local feature condition on an object)
  the set of nodes that satisfy the condition.
  This is the easy part:
  A single walk over all nodes could construct these sets in one go, in a fraction
  of a second;
* then work through all edges, where every edge is an instruction to weed out non-results
  from the earlier obtained sets.
  
How would that work, filtering along an edge in the graph?

Suppose there is an edge from node1 to node2 (in the query graph).
This edge specifies a relationship between nodes in the result nodeset of n1 and nodes 
in the result nodeset of n2.

In English: such an edge says: hey TF node in result set of n1: 
do you have a parent (or child, or older brother or younger sister)
that occurs in the result of n2?

If so: you can stay. If not: you're OUT.
So this weeds out TF nodes from the result set of n1.

But we can also reduce the nodes in the result set of n2.
Every TF node in the result set of n2 that does not figure as the parent
(or child or brother/sister) of a TF node in the result set of n1 is also out.

In this way we can take every edge, one by one, and perform the filtering.
This is also a fast operation, provided we can make the elementary relationship checks
quickly. (And we can, in TF, thanks to precomputing).

When we have done all edges, we probably have to iterate again over all edges.

Because an edge between n2 and n3 could have weeded out additional TF nodes from the 
result set of n2, and this influences the validity of the TF nodes in the result set of n1.

Probably we have to repeat until the result sets do not change anymore.

Or we can find a way to order the edges, so that we can do them in one or two passes.

Maybe we need a bit of math here.

My gut feeling is that this is all very doable and that it corresponds to people's query
needs.

In [1]:
import sys, collections
from tf.fabric import Fabric

In [2]:
ETCBC = 'hebrew/etcbc4c'
TF = Fabric( modules=ETCBC )

This is Text-Fabric 1.2.7
Api reference : https://github.com/ETCBC/text-fabric/wiki/Api
Tutorial      : https://github.com/ETCBC/text-fabric/blob/master/docs/tutorial.ipynb
Data sources  : https://github.com/ETCBC/text-fabric-data
Data docs     : https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/0_overview.html
Shebanq docs  : https://shebanq.ancient-data.org/text
Slack team    : https://shebanq.slack.com/signup
Questions? Ask shebanq@ancient-data.org for an invite to Slack
105 features found and 0 ignored


In [3]:
api = TF.load('''
    lex 
    typ code function rela det
    oslots
''')
api.makeAvailableIn(globals())

  0.00s loading features ...
   |     0.55s B oslots               from /Users/dirk/github/text-fabric-data/hebrew/etcbc4c
   |     0.00s M otext                from /Users/dirk/github/text-fabric-data/hebrew/etcbc4c
   |     0.14s B lex                  from /Users/dirk/github/text-fabric-data/hebrew/etcbc4c
   |     0.23s B typ                  from /Users/dirk/github/text-fabric-data/hebrew/etcbc4c
   |     0.04s B code                 from /Users/dirk/github/text-fabric-data/hebrew/etcbc4c
   |     0.08s B function             from /Users/dirk/github/text-fabric-data/hebrew/etcbc4c
   |     0.23s B rela                 from /Users/dirk/github/text-fabric-data/hebrew/etcbc4c
   |     0.16s B det                  from /Users/dirk/github/text-fabric-data/hebrew/etcbc4c
  5.54s All features loaded/computed - for details use loadLog()


## Freezing
Nodes are defined as datastructures.
If we are going to work with them, we have to refer to them, use them as dictionary keys and so on.
So we are going to replace them by immutable datastructures. `freeze()` is going to do just that.

In [4]:
def freeze(d):
    if type(d) is set: return frozenset(freeze(e) for e in d)
    if type(d) is list or type(d) is tuple: return tuple(freeze(e) for e in d)
    if type(d) is dict: return tuple(((e[0], freeze(e[1])) for e in sorted(d.items())))
    return d

## Compiling

We receive the nodes and edges of a query, give the nodes a number,
and replace the *from*- and *to*-nodes
of the edges by the numbers of those nodes.

We also build a list of otypes of the nodes, in the same order as the nodes themselves.

In [5]:
def compileQuery(nodes, edges):
    otypes = []
    compiledEdges = []
    nodeIndex = {}
    for (i, node) in enumerate(nodes):
        fNode = freeze(node)
        nodeIndex[fNode] = i
        otypes.append(node[0])
    for (i, (nodeFrom, nodeTo)) in enumerate(edges):
        (fNodeFrom, fNodeTo) = (freeze(nodeFrom), freeze(nodeTo))
        if fNodeFrom not in nodeIndex:
            error('From part in edge {} is not a node: {}'.format(i, nodeFrom))
            continue
        if fNodeTo not in nodeIndex:
            error('To part in edge {} is not a node: {}'.format(i, nodeTo))
            continue
        fromI = nodeIndex[fNodeFrom]
        toI = nodeIndex[fNodeTo]
        compiledEdges.append((fromI, toI))
    return (otypes, compiledEdges)

## Atoms

The first stage of running a query is to run the individual nodes as filters in their associated
object types.

A node is specified by an `otype` and a features dict.
The features specifiy any number of features, with an allowed value or set of allowed values for each of them.

Running an atom means to filter the set of all nodes in its `otype` by means of its `features`.

The result list is delivered in a list of result lists, corresponding to the nodes list.

In [6]:
def runAtom(otype, features):
    featureList = sorted(features.items())
    result = set()
    for n in F.otype.s(otype):
        good = True
        for (ft, val) in featureList:
            fval = Fs(ft).v(n)
            if type(val) is str or type(val) is int:
                if fval != val:
                    good = False
                    break
            else:
                if fval not in val:
                    good = False
                    break
        if good: result.add(n)
    return result

def runAtoms(nodes, otypes, results):
    info('Getting results for {} atoms'.format(len(nodes)))
    for node in nodes:
        results.append(runAtom(*node))
        info('.', tm=False, nl=False)
    info('\nDone')
    showResults(otypes, results)

## Edges

Once the atoms have done their work, it is time to work out the constraints posed by edges.

An edge from query node `nF` to query node `nT` means
* that the text nodes in the result of `nF` should embed a text node in the result of `nT`,
or equivalently,
* that the text nodes in the result of `nT` should be embedded in a text node in the result of `nF`.

This will reduce the amount of nodes in both `nF` and `nT`.

### Note
> In general, one pass over all edges will not be enough.
We have to repeat the process until nothing changes anymore.

The challenge is now to run the edges in an optimal sequence.

In [13]:
def runEdge(f, t, otypes, results):
    resultsF = results[f]
    resultsT = results[t]
    newResultsF = set()
    newResultsT = set()
    for n in resultsT:
        fs = set(L.u(n, otype=otypes[f])) & resultsF
        if len(fs):
            newResultsF |= fs
            newResultsT.add(n)
    results[f] = newResultsF
    results[t] = newResultsT

def runEdges(compiledEdges, otypes, results, order=None):
    info('Filtering by {} edges'.format(len(compiledEdges)))
    cedges = compiledEdges if order == None else sorted(compiledEdges, key=order)
    for (f, t) in compiledEdges:
        runEdge(f, t, otypes, results)
        info('.', tm=False, nl=False)
    info('\nDone')
    showResults(otypes, results)

## Strategy

Here we develop a strategy of running edges in a good sequence.

The basic intuition is this.

* some query nodes filter strongly, others hardly, i.e. some atom results are small compared to the total
  number of nodes in their otype, other atom results are nearly as big as the total otype.
* if an edge connects a strongly filtering node with a weakly filtering node, we expect a big reduction
* if we work within the strongest filtering query nodes, we do not have to do much work and when we reach the
  weaker filters, they will decrease rapidly
* so we postpone to touch the bigger sets as long as possible, and when we touch them, they are expected to decrease
  quickly

We are going to rank query nodes by how strong they have filtered their otype so far.

* let $o_n$ be the otype associated with query node $n$
* let $r_n$ be the current result set associated with query node $n$


Then the **query fraction** $q(n)$ is the a proportion between
the number of text nodes in the current result:
and
the total number of text nodes in the otype:

$$q(n) = {|r_n|\over |o_n|}$$

Then we rank the edges by combined query fraction of the nodes:

$$ r(f,t) = q(f)^2 + q(t)^2$$

By squaring both query fractions, we strongly give precedence to edges involving few results.

### Iteration

We can rank the edges, then run them all in the required order.
But we can also run a single edge, and then rank a new.

In [14]:
def rankEdges(edges, otypes, results):
    queryFraction = {}
    for (i, r) in enumerate(results):
        otype = otypes[i]
        (begin, end) = F.otype.sInterval(otype)
        nOtype = 1 + end - begin
        nResults = len(r)
        qf = nResults / nOtype
        queryFraction[i] = qf * qf
    return (lambda x: queryFraction[x[0]] + queryFraction[x[1]])

## Query

Here we put everything together.

In [15]:
def runQuery(nodes, edges):
    (otypes, compiledEdges) = compileQuery(nodes, edges)
    indent(reset=True)
    results = []
    runAtoms(nodes, otypes, results)
    runEdges(compiledEdges, otypes, results, order=rankEdges(compiledEdges, otypes, results))
    info('Done')
    return results

## Showing

We want to see intermediate results.
Query results are given as a list of lists.
Every individual result list corresponds to the results of a single node,
subject to more or less filtering.
Here we show the current lengths of those lists.

In [16]:
def showResults(otypes, results):
    indent(level=1)
    for i in range(len(results)):
        info('{:>2}-{:<20}: {} results'.format(
            i, otypes[i], len(results[i]),
        ))
    indent(level=0)
    info('Min-Max {:>7}-{:>7}'.format(
        min(len(r) for r in results),
        max(len(r) for r in results),
    ))

In [17]:
bk = ('book', dict())
ch = ('chapter', dict())
vs = ('verse', dict())
cl = ('clause', dict())
ca = ('clause_atom', dict())
ph = ('phrase', dict())
pa = ('phrase_atom', dict())
w = ('word', dict(lex={'JC/', '>JN/'}))
nodes = [bk, ch, vs, cl, ca, ph, pa, w]
edges = [
    [bk, ch],
    [ch, vs],
    [vs, cl],
    [cl, ca],
    [ca, ph],
    [ph, pa],
    [pa, w],
]

In [18]:
results = runQuery(nodes, edges)

  0.00s Getting results for 8 atoms
........  0.80s 
Done
   |    1m 12s  0-book                : 39 results
   |    1m 12s  1-chapter             : 929 results
   |    1m 12s  2-verse               : 23213 results
   |    1m 12s  3-clause              : 88000 results
   |    1m 12s  4-clause_atom         : 90562 results
   |    1m 12s  5-phrase              : 253174 results
   |    1m 12s  6-phrase_atom         : 267515 results
   |    1m 12s  7-word                : 926 results
  0.81s Min-Max      39- 267515
  0.81s Filtering by 7 edges
.......  7.33s 
Done
   |    1m 18s  0-book                : 39 results
   |    1m 18s  1-chapter             : 929 results
   |    1m 18s  2-verse               : 23207 results
   |    1m 18s  3-clause              : 87950 results
   |    1m 18s  4-clause_atom         : 90007 results
   |    1m 18s  5-phrase              : 252590 results
   |    1m 18s  6-phrase_atom         : 923 results
   |    1m 18s  7-word                : 926 results
  7.34s M

In [116]:
results = runQuery(nodes, edges)

  0.00s Getting results for 8 atoms
........  0.78s 
Done
   |   20m 56s  0-book                : 39 results
   |   20m 56s  1-chapter             : 929 results
   |   20m 56s  2-verse               : 23213 results
   |   20m 56s  3-clause              : 88000 results
   |   20m 56s  4-clause_atom         : 90562 results
   |   20m 56s  5-phrase              : 253174 results
   |   20m 56s  6-phrase_atom         : 267515 results
   |   20m 56s  7-word                : 926 results
  0.78s Min-Max      39- 267515
  0.78s Filtering by 7 edges
.......  7.38s 
Done
   |   21m 02s  0-book                : 39 results
   |   21m 02s  1-chapter             : 929 results
   |   21m 02s  2-verse               : 23207 results
   |   21m 02s  3-clause              : 88000 results
   |   21m 02s  4-clause_atom         : 90088 results
   |   21m 02s  5-phrase              : 253174 results
   |   21m 02s  6-phrase_atom         : 923 results
   |   21m 02s  7-word                : 926 results
  7.38s M